In [174]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.signal as ss
import plotly
from math import sqrt, sin, cos, tan, asin, acos, atan, atan2
from dataFilter import butter_highpass_filter, butter_lowpass_filter
from selfpkg import readDB, MinToMS

In [175]:
class Kalman:
    def __init__(self, Q_angle=1, Q_bias=0.01, R_measure=0.0001, \
                            angle=0.0, bias=0.0, \
                            P00=0.0, P01=0.0, P10=0.0, P11=0.0):
        self.Q_angle = Q_angle
        self.Q_bias = Q_bias
        self.R_measure = R_measure
        
        self.angle = angle
        self.bias = bias
        
        self.P00 = P00
        self.P01 = P01
        self.P10 = P10
        self.P11 = P11       
    
    #The angle should be in degrees and the rate should be in degrees per second and the delta time in seconds
    def getAngle(self, newAngle, newRate, dt):
        #KasBot V2  -  Kalman filter module - http://www.x-firm.com/?page_id=145
        #Modified by Kristian Lauszus
        #See my blog post for more information: http://blog.tkjelectronics.dk/2012/09/a-practical-approach-to-kalman-filter-and-how-to-implement-it

        #Discrete Kalman filter time update equations - Time Update ("Predict")
        #Update xhat - Project the state ahead
        #Step 1
        rate = newRate - self.bias
        self.angle += dt * rate

        #Update estimation error covariance - Project the error covariance ahead
        #Step 2 
        self.P00 += dt * (dt * self.P11 - self.P01 - self.P10 + self.Q_angle)
        self.P01 -= dt * self.P11
        self.P10 -= dt * self.P11
        self.P11 += self.Q_bias * dt

        #Discrete Kalman filter measurement update equations - Measurement Update ("Correct")
        #Calculate Kalman gain - Compute the Kalman gain
        #Step 4
        S = self.P00 + self.R_measure # Estimate error
        #Step 5
        # Kalman gain - This is a 2x1 vector
        K0 = self.P00 / S
        K1 = self.P10 / S

        #Calculate angle and bias - Update estimate with measurement zk (newAngle)
        #Step 3
        y = newAngle - self.angle #Angle difference
        #Step 6
        self.angle += K0 * y;
        self.bias += K1 * y;

        #Calculate estimation error covariance - Update the error covariance
        #Step 7
        P00_temp = self.P00
        P01_temp = self.P01

        self.P00 -= K0 * P00_temp
        self.P01 -= K0 * P01_temp
        self.P10 -= K1 * P00_temp
        self.P11 -= K1 * P01_temp

        return self.angle
    
    def setAngle(self, angle):
        self.angle = angle
    def getRate(self):
        return self.rate
    
    def setQangle(self, Q_angle):
        self.Q_angle = Q_angle
    def setQbias(self, Q_bias):
        self.Q_bias = Q_bias
    def setRmeasure(self, R_measure):
        self.R_measure = R_measure
    
    def getQangle(self):
        return self.Q_angle
    def getQbias(self):
        return self.Q_bias
    def getRmeasure(self):
        return self.R_measure

In [176]:
radToDeg = 180 / 3.14159

In [183]:
def IMUdataFusion(df, i):    
    #acc
    accX = df.accX[i] / 10
    accY = df.accY[i] / 10
    accZ = df.accZ[i] / 10
    #magnetic sensor
    compassX = df.compassX[i]
    compassY = df.compassY[i]
    compassZ = df.compassZ[i]
    #gyroscope
    rotationalSpeedX = df.rotationalSpeedX[i]
    rotationalSpeedY = df.rotationalSpeedY[i]
    rotationalSpeedZ = df.rotationalSpeedZ[i]
    
    #A_roll & A_pitch from acceleration
    roll = atan2(accY, accZ) * radToDeg
    pitch = atan2(-accX, sqrt(accY ** 2 + accZ ** 2)) * radToDeg
    
    norm = sqrt(accX ** 2 + accY ** 2 + accZ ** 2)
    pitchA = -asin(accX / norm)
    s = accY / cos(pitchA) / norm
    rollA = asin(round(s, 5))
    
    #M_yaw from magnetic
    norm = sqrt(compassX ** 2 + compassY ** 2 + compassZ ** 2)
    mx = compassX / norm
    my = - compassY / norm
    mz = compassZ / norm
    
    Mx = mx * cos(pitchA) + mz * sin(pitchA)
    My = mx * sin(rollA) * sin(pitchA) + my * cos(rollA) - mz * sin(rollA) * cos(pitchA)
    
    yaw = atan2(-My,Mx) * radToDeg
    #fix yaw
    if yaw > 360:
        yaw -= 360
    elif yaw < 0:
        yaw += 360
    
    temp = 0.02
    #kalman filter
    KFAngle[0] = KFilter[0].getAngle(roll, rotationalSpeedX, temp)
    KFAngle[1] = KFilter[1].getAngle(pitch, rotationalSpeedY, temp)
    KFAngle[2] = KFilter[2].getAngle(yaw, rotationalSpeedZ, temp)

    #complimentary filter
    #CF_angle = 0.98 * (CF_angle + gyro_data * delta t) + 0.02 * acc_data
#     gyro_newangle = rotationalSpeedZ * temp
#     yaw_gyro += gyro_newangle
#     CFAngle[2] = 0.98 * (CFAngle[2] + gyro_newangle) + 0.02 * yaw
    
    return KFAngle#, CFAngle

In [184]:
def IMUwithKalmanFilter(df):
    
    #acc
    accX = df.accX[0] / 10
    accY = df.accY[0] / 10
    accZ = df.accZ[0] / 10
    #magnetic sensor
    compassX = df.compassX[0]
    compassY = df.compassY[0]
    compassZ = df.compassZ[0]
    #gyroscope
#     rotationalSpeedX_arr = butter_highpass_filter(df.rotationalSpeedX, 1, 50, 5)
#     rotationalSpeedY_arr = butter_highpass_filter(df.rotationalSpeedY, 1, 50, 5)
#     rotationalSpeedZ_arr = butter_highpass_filter(df.rotationalSpeedZ, 1, 50, 5)
    
    rotationalSpeedX = df.rotationalSpeedX[0]
    rotationalSpeedY = df.rotationalSpeedY[0]
    rotationalSpeedZ = df.rotationalSpeedZ[0]

    KFilter[0].setAngle(atan2(accY, accZ) * radToDeg)
    KFilter[1].setAngle((atan2(-accX, sqrt(accY ** 2 + accZ ** 2))) * radToDeg)

    ##A_roll & A_pitch from acceleration
    roll = atan2(accY, accZ) * radToDeg
    pitch = atan2(-accX, sqrt(accY ** 2 + accZ ** 2)) * radToDeg

    norm = sqrt(accX ** 2 + accY ** 2 + accZ ** 2)
    pitchA = -asin(accX / norm)
    s = accY / cos(pitchA) / norm
    rollA = asin(round(s,5))

    #M_yaw from magnetic
    norm = sqrt(compassX ** 2 + compassY ** 2 + compassZ ** 2)
    mx = compassX / norm
    my = - compassY / norm
    mz = compassZ / norm

    Mx = mx * cos(pitchA) + mz * sin(pitchA)
    My = mx * sin(rollA) * sin(pitchA) + my * cos(rollA) - mz * sin(rollA) * cos(pitchA)

    yaw = atan2(-My,Mx) * radToDeg

    #fix yaw
    if yaw > 360:
        yaw -= 360
    elif yaw < 0:
        yaw += 360

    KFilter[0].setAngle(roll)
    KFilter[1].setAngle(pitch)
    KFilter[2].setAngle(yaw)
    # yaw_gyro = yaw
#     CFAngle[2] = yaw
    # roll = atan2(accY, accZ) * radToDeg
    # pitch = atan2(-accX, sqrt(accY ** 2 + accZ ** 2)) * radToDeg
    KRes0.append(roll)
    KRes1.append(pitch)
    KRes2.append(yaw)

    for i in range(1, len(df.index)):
        K = IMUdataFusion(df, i)
        KRes0.append(K[0])
        KRes1.append(K[1])
        KRes2.append(K[2])
#         CRes.append(C[2])

In [ ]:
#function for debouncing orientation
def orientationTrans(dfOrientation, start, end, threshold, df):
    startIndex = df.index[df['time'] == start].tolist()[0] #get the index
    endIndex = df.index[df['time'] == end].tolist()[0]
    prev = dfOrientation[startIndex]
    
    for i in range(startIndex, endIndex+1):
        diff = dfOrientation[i] - prev
        prev = dfOrientation[i]
        if(diff > threshold or diff < -threshold):
            if(diff > 0):
                dfOrientation[i] = threshold * 2 - diff
            else:
                dfOrientation[i] = - (threshold * 2 + diff)
        else:
            dfOrientation[i] = diff
    return dfOrientation

In [185]:
kx = Kalman()
ky = Kalman()
kz = Kalman()
KFilter = [kx, ky, kz]
KFAngle = [0, 0, 0]

KRes0 = []
KRes1 = []
KRes2 = []
csv_database = 'sqlite:///KFtest04.db'
df = readDB(csv_database)
# df = pd.read_csv('/Users/meng/Documents/dataPreprocessing/venv/source/sa_run08_trim.csv')
IMUwithKalmanFilter(df)

In [186]:
from directionCalculation import getDirectionByRotation, getDirection

In [187]:
Angles_rotation = getDirectionByRotation(df)
Angles = getDirection(df)

In [188]:
traceRoll = go.Scattergl(
    x=df.time/60000,
    y=KRes0,#[x / 3.14 for x in KRes0],
    name = 'Kalman roll',
    line = dict(
        color = ('rgb(255, 0, 0)'),
        width = 1)
)

tracePitch = go.Scattergl(
    x=df.time/60000,
    y=KRes1,#[ x/3.14 for x in KRes1],
    name = 'Kalman pitch',
    line = dict(
        color = ('rgb(0, 255, 0)'),
        width = 1)
)

traceYaw = go.Scattergl(
    x=df.time/60000,
    y=KRes2,#[ x / 3.14 for x in KRes2],
    name = 'Kalman yaw',
    line = dict(
        color = ('rgb(0, 0, 255)'),
        width = 1)
)

# trace1 = go.Scattergl(
#     x=df.time/60000,
#     y=CRes,
#     name = 'complimentary yaw',
#     line = dict(
#         color = ('rgb(127, 127, 127)'),
#         width = 1)
# )

trace2 = go.Scattergl(
    x = df.time/60000,
    y = Angles_rotation,
    name = 'angle rotation',
    line = dict(
        color = ('rgb(0, 0, 0)'),
        width = 1
    )
)

trace3 = go.Scattergl(
    x = df.time/60000,
    y = [-x for x in Angles],
    name = 'angle',
    line = dict(
        color = ('rgb(225, 225, 0)'),
        width = 1
    )
)

traceX = go.Scattergl(
    x = df.time/60000,
    y = df.orientationX,
    name = 'orientation X',
    line = dict(
        color = ('rgb(255, 0, 0)'),
        width = 1
    )
)

traceY = go.Scattergl(
    x = df.time/60000,
    y = df.orientationY,
    name = 'orientationY',
    line = dict(
        color = ('rgb(0, 0, 255)'),
        width = 1
    )
)

traceZ = go.Scattergl(
    x = df.time/60000,
    y = df.orientationZ,
    name = 'orientationZ',
    line = dict(
        color = ('rgb(127, 127, 127)'),
        width = 1
    )
)

data = [traceYaw, traceRoll, tracePitch, trace2, trace3, traceX, traceY, traceZ]

layout = dict(title = 'testAngle',
                   xaxis = dict(title = 'Time(minutes)'),
                   yaxis = dict(title = 'Angles(degrees)'))

fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, filename='testAngle')

'file:///Users/meng/Documents/dataPreprocessing/venv/source/testAngle.html'

In [70]:
df['roll'] = KRes0
df['pitch'] = KRes1
df['yaw'] = KRes2

In [93]:
df.to_csv('testKalman02.csv')